## Load Module Files from GitHub

In [1]:
!rm /content/*.py*

rm: cannot remove '/content/*.py*': No such file or directory


In [2]:
queries = []

filenames = [
             "GraphAttentionHead.py",
             "MultiHeadGraphAttention.py",
             "MultiHeadGraphAttentionLSTMCell.py",
             "mhga_utils.py"
]

for fn in filenames:
  queries.append(f"https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/{fn}")

queries.append("https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/imports.py")

In [3]:
queries

['https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/GraphAttentionHead.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/MultiHeadGraphAttention.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/MultiHeadGraphAttentionLSTMCell.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/mhga_utils.py',
 'https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/imports.py']

In [4]:
import subprocess
# Download and import my module files from GitHub
result = subprocess.run(["wget", "--no-cache", "--backups=1"] + queries, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
print(result.stderr.decode("utf-8"))

--2022-07-29 01:45:56--  https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/GraphAttentionHead.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2327 (2.3K) [text/plain]
Failed to rename GraphAttentionHead.py to GraphAttentionHead.py.1: (2) No such file or directory
Saving to: ‘GraphAttentionHead.py’

     0K ..                                                    100% 28.5M=0s

2022-07-29 01:45:57 (28.5 MB/s) - ‘GraphAttentionHead.py’ saved [2327/2327]

--2022-07-29 01:45:57--  https://raw.githubusercontent.com/NuoWenLei/MultimodalFusionWithUnorthodoxInputs/main/GraphAttentionLSTM/MultiHeadGraphAttention.py
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response.

## Load and Create Google Drive Linked Environment

In [5]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [6]:
folder_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/"
flight_data_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/flight_data/aggregated_flight_map_data.csv"
covid_data_path = "/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/US_time_series_covid_cases.csv"

## Load Sequential Data from Google Drive

In [7]:
from mhga_utils import *

In [8]:
NUM_DAYS_PER_SAMPLE = 7

In [9]:
(formatted_X, formatted_adj_mat, formatted_y_infection, formatted_y_death) = load_sequential_data(covid_data_path, flight_data_path, num_days_per_sample = NUM_DAYS_PER_SAMPLE)

Generating Sequential Data...


100%|██████████| 793/793 [00:24<00:00, 32.07it/s]


## Scale and Split Train Test

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize

In [11]:
# MinMax Scale by column
X_max = formatted_X.reshape(-1, formatted_X.shape[-1]).max(axis = 0)
X_min = formatted_X.reshape(-1, formatted_X.shape[-1]).min(axis = 0)
X_scaled = (formatted_X - X_min) / (X_max - X_min)

mat_max = formatted_adj_mat.reshape(-1,).max(axis = 0)
mat_min = formatted_adj_mat.reshape(-1,).min(axis = 0)
mat_scaled = (formatted_adj_mat - mat_min) / (mat_max - mat_min)

X_scaled.shape, mat_scaled.shape

((793, 7, 49, 5), (793, 7, 49, 49))

In [12]:
X_train, X_test, adj_mat_train, adj_mat_test, y_train, y_test = train_test_split(X_scaled, mat_scaled, formatted_y_infection, train_size = .8)

## Load Model from Module

In [13]:
MODEL_ARCHITECTURE = (16, 16, 16)
NUM_HEADS = 8
SEQUENCE_LENGTH = 49
D_MODEL = 32

In [14]:
gat_lstm = create_graph_attention_lstm_model(
    layer_units = MODEL_ARCHITECTURE,
    num_heads = NUM_HEADS,
    input_shape_nodes = (NUM_DAYS_PER_SAMPLE, SEQUENCE_LENGTH, 5),
    input_shape_edges = (NUM_DAYS_PER_SAMPLE, SEQUENCE_LENGTH, SEQUENCE_LENGTH),
    sequence_length = SEQUENCE_LENGTH,
    hidden_size = D_MODEL,
    residual = True,
    use_bias = True,
    seq_wise_output = True,
    name = "GraphAttentionLSTMModel"
)

In [15]:
gat_lstm.summary()

Model: "GraphAttentionLSTMModel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 7, 49, 5)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 7, 49, 49)]  0           []                               
                                                                                                  
 rnn (RNN)                      (None, 7, 49, 32)    87232       ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 rnn_1 (RNN)                    (None, 7, 49, 32)    142528      ['rnn[0][0]

## Train Model

In [16]:
optimizer = tf.keras.optimizers.Adam()
BATCH_SIZE = 16
EPOCHS = 20

In [17]:
@tf.function
def loss_func(y_true, y_pred):
  squared_diff = tf.square(y_true - y_pred)
  mse = tf.reduce_mean(squared_diff, axis=-1)
  squared_diff_mean = tf.square(tf.reduce_mean(y_true, axis = -1) - tf.reduce_mean(y_pred, axis = -1))
  return mse + squared_diff_mean

In [18]:
gat_lstm.compile(
    loss = loss_func,
    optimizer = optimizer,
    metrics = ["mse", "mae"]
)

In [19]:
gat_lstm.fit(
    x = (X_train, adj_mat_train),
    y = y_train,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS)

Epoch 1/20
40/40 [==============================] - 490s 4s/step - loss: 0.0113 - mse: 0.0068 - mae: 0.0653
Epoch 2/20
40/40 [==============================] - 174s 4s/step - loss: 0.0012 - mse: 0.0010 - mae: 0.0243
Epoch 3/20
40/40 [==============================] - 173s 4s/step - loss: 3.1288e-04 - mse: 2.8582e-04 - mae: 0.0128
Epoch 4/20
40/40 [==============================] - 174s 4s/step - loss: 1.8998e-04 - mse: 1.6431e-04 - mae: 0.0096
Epoch 5/20
40/40 [==============================] - 173s 4s/step - loss: 1.5674e-04 - mse: 1.3184e-04 - mae: 0.0086
Epoch 6/20
40/40 [==============================] - 173s 4s/step - loss: 1.2751e-04 - mse: 1.0673e-04 - mae: 0.0078
Epoch 7/20
40/40 [==============================] - 173s 4s/step - loss: 1.1392e-04 - mse: 9.2276e-05 - mae: 0.0073
Epoch 8/20
40/40 [==============================] - 173s 4s/step - loss: 9.9148e-05 - mse: 7.9310e-05 - mae: 0.0068
Epoch 9/20
40/40 [==============================] - 174s 4s/step - loss: 8.9888e-05 - ms

In [20]:
eval_res = gat_lstm.evaluate(
    x = (X_test, adj_mat_test),
    y = y_test,
    batch_size = BATCH_SIZE)

10/10 [==============================] - 34s 494ms/step - loss: 3.6981e-05 - mse: 3.3785e-05 - mae: 0.0043


In [21]:
eval_res

[3.6980938602937385e-05, 3.3784723200369626e-05, 0.004338302183896303]

In [22]:
gat_lstm.save_weights("/content/gdrive/MyDrive/Inspirit AI/Inspirit_Research_Project/graph_attention_weights/graph_attention_weights_with_metric/model_weights")